In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://www.calendarr.com/united-kingdom/observances-2024/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, "html.parser")

# List to hold the formatted events
events = []

# Find all the tags that contain class "holiday-month"
months = soup.find_all("span", class_="holiday-month")

# Loop through each month span
for month_tag in months:
    # Get the name of the month
    month = month_tag.text.strip()
    
    # Get the next tags which are expected to contain the holiday information
    next_tags = month_tag.find_all_next("ul", class_="list-holidays")

    # Iterate over each next tag
    for next_tag in next_tags:
        # Check if the tag belongs to the current month
        if next_tag.find_previous("span", class_="holiday-month") != month_tag:
            break

        # Loop through all child list-holiday-box elements within the next_tag
        for box in next_tag.find_all("li", class_="list-holiday-box"):
            # Extract day, weekday, and name from the children tags
            day = box.find("span", class_="holiday-day").text.strip()
            weekday = box.find("span", class_="holiday-weekday").text.strip()
            
            # Event name can be in an <a> or <span> tag with class "holiday-name"
            name_tag = box.find("a", class_="holiday-name") or box.find("span", class_="holiday-name")
            name = name_tag.text.strip() if name_tag else "No name found"

            # Append the formatted event to the list
            events.append(f"{month}, {day}, {weekday}, {name}")

# Print the formatted events
for event in events:
    print(event)


January, 01, Mon, New Year's Day
January, 01, Mon, Mexican Independence Day
January, 01, Mon, Dry January
January, 01, Mon, Veganuary
January, 01, Mon, International Self Care Day
January, 01, Mon, National Bloody Mary Day
January, 02, Tue, World Introvert Day
January, 04, Thu, World Braille Day
January, 04, Thu, National Trivia Day
January, 04, Thu, World Hypnotism Day
January, 05, Fri, National Whipped Cream Day
January, 06, Sat, Epiphany
January, 06, Sat, National Bean Day
January, 07, Sun, Orthodox Christmas
January, 11, Thu, National Milk Day
January, 12, Fri, National Pharmacist Day
January, 14, Sun, World Logic Day
January, 15, Mon, Brew Monday
January, 16, Tue, National Nothing Day
January, 17, Wed, Museum Selfie Day
January, 17, Wed, Guru Gobind Singh Jayanti
January, 18, Thu, National Thesaurus Day
January, 19, Fri, National Popcorn Day
January, 20, Sat, World Religion Day
January, 21, Sun, National Squirrel Appreciation Day
January, 21, Sun, World Snow Day
January, 23, Tue, 

In [1]:
pip install mysql-connector-python

In [6]:
import mysql.connector
from mysql.connector import errorcode

# Function to establish a connection to MySQL
def connect_to_mysql():
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host="", # your database host
            user="", # your database user
            password="", # your database password
            database=""  # your database name
        )
        print("Connected to MySQL!")
        return connection
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your username or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

def map_month(month_name):
    months = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
        'May': '05',
        'June': '06',
        'July': '07',
        'August': '08',
        'September': '09',
        'October': '10',
        'November': '11',
        'December': '12'
    }
    return months.get(month_name, '01')  # Default to January if not found

# Insert data into MySQL
def insert_data_into_mysql(events):
    connection = connect_to_mysql()
    if connection:
        try:
            cursor = connection.cursor()

            # SQL query to insert data
            insert_query = ("INSERT INTO event "
                            "(description, title, start_date, end_date, created_on, updated_on, image_name, person_id) "
                            "VALUES (%s, %s, %s, %s, NOW(), NOW(), '', 1)")

            # Insert each event into the database
            for event in events:
                event_data = event.split(", ")
                description = "Dummy description"  # You can change this to whatever description you want
                title = event_data[3]  # Event name
                month = map_month(event_data[0])  # Map month name to numerical value
                day = event_data[1]
                start_date = f"2024-{month}-{day}"  # Assuming year is 2024
                end_date = start_date  # Assuming events are single-day
                event_values = (description, title, start_date, end_date)
                cursor.execute(insert_query, event_values)

            # Commit changes
            connection.commit()
            print("Data inserted successfully!")

        except mysql.connector.Error as error:
            print("Failed to insert data into MySQL table:", error)

        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")


# Call the function to insert data
insert_data_into_mysql(events)


Connected to MySQL!
Data inserted successfully!
MySQL connection is closed


In [12]:
import requests

def search_images(query, num_images=1):
    # Replace "YOUR_API_KEY" with your Pixabay API key
    api_key = "44095299-6aba851af64e452adb080a6b9"
    url = f"https://pixabay.com/api/?key={api_key}&q={query}&per_page={num_images}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        image_urls = [item['largeImageURL'] for item in data.get('hits', [])]
        return image_urls
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return []

# Example usage:
query = "America Independence Day"
image_urls = search_images(query, num_images=4)

for url in image_urls:
    print(url)


https://pixabay.com/get/gc44676f53310854b8cee2cdaf131fba17e8bdb9772ec94324993fe59ad8900947320d6b6183a4982afc4a400b6216cab7a87a0402e014814d82538415254ac23_1280.jpg
https://pixabay.com/get/g76ceb7bd2c0d2fe5aa6d62b88a90bb0ace93f54dca6aaffc5219347c1adee42a08d0229358d40d4269db1a2100096f0bc3d472116d48797469735aacbc08dbeb_1280.jpg
https://pixabay.com/get/g1e7fb5d686d0216a863c91138d5f356bde292680225c9676a7956ba80785aa9ff90be49df864b6750500a8223ed451f927e2b094882ea25271ae90a111683836_1280.jpg
https://pixabay.com/get/g99a4bb20b69d73c0b35f2c9dda4d2a7bf5af7d8ac26eafc0637183c451591d20231e61463ccc796911ebbf1a4b50b576b11b84b1894f66ad72edf76cf469caaa_1280.jpg


In [7]:
import requests
from urllib.parse import quote

def search_image(query):
    # Replace "YOUR_API_KEY" with your Pixabay API key
    api_key = ""
    encoded_query = quote(query)  # URL encode the query
    url = f"https://pixabay.com/api/?key={api_key}&q={encoded_query}&per_page=3"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if data['totalHits'] > 0:
            image_url = data['hits'][0]['largeImageURL']
            return image_url
        else:
            print("No images found for the query:", query)
            return None
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

# Example usage:
query = "Independence Day"
image_url = search_image(query)

if image_url:
    print("Image URL:", image_url)
    # You can display the image using IPython.display.Image


Error: 400 Client Error: Bad Request for url: https://pixabay.com/api/?key=&q=Independence%20Day&per_page=3


In [3]:
import mysql.connector
from mysql.connector import errorcode
import requests
from urllib.parse import quote

# Function to establish a connection to MySQL
def connect_to_mysql():
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host="localhost", # your database host
            user="root", # your database user
            password="abcabc<<<123", # your database password
            database="poc-school"  # your database name
        )
        print("Connected to MySQL!")
        return connection
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your username or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

# Function to read all events from MySQL
def read_events_from_mysql():
    connection = connect_to_mysql()
    if connection:
        try:
            cursor = connection.cursor()

            # Select all events from the event table
            select_query = "SELECT id, title FROM event"
            cursor.execute(select_query)
            events = cursor.fetchall()

            return events

        except mysql.connector.Error as error:
            print("Failed to read events from MySQL:", error)

        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")





events = read_events_from_mysql()

Connected to MySQL!
MySQL connection is closed


In [4]:
print(events)

[(1, "New Year's Day"), (2, 'Mexican Independence Day'), (3, 'Dry January'), (4, 'Veganuary'), (5, 'International Self Care Day'), (6, 'National Bloody Mary Day'), (7, 'World Introvert Day'), (8, 'World Braille Day'), (9, 'National Trivia Day'), (10, 'World Hypnotism Day'), (11, 'National Whipped Cream Day'), (12, 'Epiphany'), (13, 'National Bean Day'), (14, 'Orthodox Christmas'), (15, 'National Milk Day'), (16, 'National Pharmacist Day'), (17, 'World Logic Day'), (18, 'Brew Monday'), (19, 'National Nothing Day'), (20, 'Museum Selfie Day'), (21, 'Guru Gobind Singh Jayanti'), (22, 'National Thesaurus Day'), (23, 'National Popcorn Day'), (24, 'World Religion Day'), (25, 'National Squirrel Appreciation Day'), (26, 'World Snow Day'), (27, 'National Pie Day'), (28, 'Tu Bishvat'), (29, 'National Peanut Butter Day'), (30, 'International Day of Education'), (31, 'Burns Night'), (32, "st dwynwen's day"), (33, 'Mahayana New Year'), (34, 'National Irish Coffee Day'), (35, 'National Fun at Work Da

In [27]:
import requests
from urllib.parse import quote
import os
import requests
from urllib.parse import quote

def search_image(query):
    # Replace "YOUR_API_KEY" with your Pixabay API key
    api_key = "44095299-6aba851af64e452adb080a6b9"
    encoded_query = quote(query)  # URL encode the query
    url = f"https://pixabay.com/api/?key={api_key}&q={encoded_query}&per_page=4"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if data['totalHits'] > 0:
            image_url = data['hits'][0]['largeImageURL']
            return image_url
        else:
            print("No images found for the query:", query)
            return None
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None


# Function to download and rename the image
def download_image(url, path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(path, 'wb') as out_file:
            for chunk in response.iter_content(chunk_size=8192):
                out_file.write(chunk)
        return path
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return None


# Function to update image names in MySQL
def update_image_names_in_mysql(events, image_paths):
    connection = connect_to_mysql()
    if connection:
        try:
            cursor = connection.cursor()

            # Update each event with the corresponding image path
            for event, path in zip(events, image_paths):
                event_id, _ = event  # Extract event ID from the tuple
                print(path)
                update_query = ("UPDATE event "
                                "SET image_name = %s "
                                "WHERE id = %s")
                update_values = (path, event_id)
                cursor.execute(update_query, update_values)

            # Commit changes
            connection.commit()
            print("Image names updated successfully!")

        except mysql.connector.Error as error:
            print("Failed to update image names in MySQL:", error)

        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")

# Main directory to store images
image_dir = "D:\\Hamza Nasir\\poc-school\\backend\\src\\main\\resources\\images"
os.makedirs(image_dir, exist_ok=True)


# Read all events from the database
events = read_events_from_mysql()

# Search for image URLs using Pixabay API
image_paths = []
for event in events[600:603]:
    print(event)
    event_id, title = event
    image_url = search_image(event[1])
    if image_url:
        print(image_url)
        # Define the image path using the event ID
        image_path = os.path.join(image_dir, f"{event_id}.jpg")
        downloaded_path = download_image(image_url, image_path)
        print(downloaded_path)
        if downloaded_path:
            image_name = os.path.basename(downloaded_path)
            image_paths.append(image_name)
        else:
            image_paths.append("")
    else:
        image_paths.append("")

# Update image names in the database
update_image_names_in_mysql(events[600:603], image_paths)

Connected to MySQL!
MySQL connection is closed
(601, 'Kwanzaa')
https://pixabay.com/get/g781d6c69e622c4bd2afeffba9a2a992d648a0f075253b931059a7ab4597b5f2ffca452d2707bee8c75c64b087983b515a98f76513635ef3bd4b129a1860c3682_1280.png
D:\Hamza Nasir\poc-school\backend\src\main\resources\images\601.jpg
(602, 'International Day of Epidemic Preparedness')
https://pixabay.com/get/geb7f4a00a2809c963494d702b08885ae8e6723c8b203dfd0db048a73795112366ea3a205868a4ea6a791a78f75f51194d2b4d80d5cd0a88d7ab8e78826ff6dcc_1280.jpg
D:\Hamza Nasir\poc-school\backend\src\main\resources\images\602.jpg
(603, "New Year's Eve")
https://pixabay.com/get/g78cf420a1cd5b953b0eb01e7aaa85cbb5596e08d32c14a80ae2a535099d0182c498b7617ecc2c03beea81d65350cd5cd69aea7cde0228651d028e81260e56edd_1280.jpg
D:\Hamza Nasir\poc-school\backend\src\main\resources\images\603.jpg
Connected to MySQL!
601.jpg
602.jpg
603.jpg
Image names updated successfully!
MySQL connection is closed
